**Connect google drive**

In [0]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun May  3 14:02:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': Operation not supported
 2019.csv
'AISHWARYA KORE latest cover letter (1).pdf'
'AISHWARYA KORE latest cover letter (2).pdf'
'AISHWARYA KORE latest cover letter (3).pdf'
'AISHWARYA KORE latest cover letter (4).pdf'
'AISHWARYA KORE latest cover letter (5).pdf'
'AISHWARYA KORE latest cover letter.pdf'
'Aishwarya Kore- Smart Course - Internship Application.gdoc'
'aishwarya resume latest (1).pdf'
'aishwarya resume latest2 (1).pdf'
'aishwarya resume latest (2).pdf'
'aishwarya resume latest2.pdf'
'aishwarya resume latest (3).pdf'
'aishwarya resume latest (4).pdf'
'aishwarya resume latest (5).pdf'
'aishwarya resume latest.pdf'
'aishwarya resume new format.docx'
 Big_data_tutorial_1.pptx.gslides
'CHARTWELLS - WSQ (CORE MENU).gdoc'
'CHARTWELLS - WSQ (CORE MENU).pdf'
'Colab Notebooks'
'Final Project.zip'
'General

**1) Clone the Darknet**



In [0]:
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


**2) Compile Darknet using Nvidia GPU**


In [0]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

[Errno 20] Not a directory: 'darknet'
/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused v

**3) Configure Darknet network for training YOLO V3**

In [0]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [0]:
!sed -i 's/batch=1/batch=2/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=2/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [0]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [0]:
!echo "follow" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [0]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-05-03 14:14:25--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M   548KB/s    in 7m 26s  

2020-05-03 14:21:54 (355 KB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [0]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
replace data/obj/img1.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/obj/img1.png       
  inflating: data/obj/img1.txt       
  inflating: data/obj/img2.png       
  inflating: data/obj/img2.txt       
  inflating: data/obj/img3.png       
  inflating: data/obj/img3.txt       
  inflating: data/obj/img4.png       
  inflating: data/obj/img4.txt       


In [0]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.868638 0.085489 0.168413 0.044994

['0', '0.868638', '0.085489', '0.168413', '0.044994']
0 data/obj/img1.txt
0 0.868638 0.085489 0.168413 0.044994
0 0.869760 0.199381 0.170659 0.044432

['0', '0.869760', '0.199381', '0.170659', '0.044432']
0 data/obj/img1.txt
0 0.869760 0.199381 0.170659 0.044432
0 0.870509 0.298931 0.161677 0.044432

['0', '0.870509', '0.298931', '0.161677', '0.044432']
0 data/obj/img1.txt
0 0.870509 0.298931 0.161677 0.044432
0 0.870883 0.372328 0.172904 0.047244

['0', '0.870883', '0.372328', '0.172904', '0.047244']
0 data/obj/img1.txt
0 0.870883 0.372328 0.172904 0.047244
0 0.872380 0.486220 0.172904 0.044432

['0', '0.872380', '0.486220', '0.172904', '0.044432']
0 data/obj/img1.txt
0 0.872380 0.486220 0.172904 0.044432
0 0.871632 0.625422 0.174401 0.046119

['0', '0.871632', '0.625422', '0.174401', '0.046119']
0 data/obj/img1.txt
0 0.871632 0.625422 0.174401 0.046119
0 0.871632 0.739314 0.159431 0.043307

['0', '0.871632', '0.739314', '0.159431', '0.043307']
0

In [0]:
import glob
images_list = glob.glob("data/obj/*.png")
print(images_list)

['data/obj/img1.png', 'data/obj/img4.png', 'data/obj/img3.png', 'data/obj/img2.png']


In [0]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [0]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 1878: 1.001195, 1.324693 avg loss, 0.001000 rate, 0.445461 seconds, 3756 images, 0.292314 hours left
Loaded: 0.000040 seconds
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.000000, GIOU: 0.000000), Class: 0.000000, Obj: 0.000000, No Obj: 0.000002, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.804042, GIOU: 0.798047), Class: 0.983149, Obj: 0.842565, No Obj: 0.001914, .5R: 1.000000, .75R: 1.000000, count: 3, class_loss = 0.096801, iou_loss = 0.443887, total_loss = 0.540688 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.904487, GIOU: 0.903660), Class: 0.996301, Obj: 0.893540, No Obj: 0.000741, .5R: 1.000000, .75R: 1.000000, count: 3, class_loss = 0.042625, iou_loss = 0.170937, total_loss = 0.213562 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 